In [1]:
import ee, datetime
import pandas as pd
import numpy as np
import folium
import geehydro
import datetime as dt
import IPython.display


In [2]:

ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


Selecting the area of Interest

In [3]:
sarapangi = folium.Map(location=[23.244061109840693,77.47089584472357], zoom_start=10)


In [4]:
sarapangi

Selecting the satellite and AOI

In [5]:
sentinel = ee.ImageCollection("COPERNICUS/S2_SR").\
               filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)).\
               filterDate("2015-01-01","2023-01-01")
sarapangi_AOI = ee.Geometry.Polygon([[77.47025211456,23.244997616477278],[77.4703594029206,23.244248411693995],[77.47089584472357,23.244061109840693],[77.4707027256745,23.24396252981228],[77.47074564101874,23.243775227557578],[77.47029502990424,23.243647073231774],[77.47043450477301,23.243499202702868],[77.47086365821539,23.243440054445415],[77.47074564101874,23.24331189979763],[77.47107823493658,23.243035873984248],[77.47135718467413,23.243213319215556],[77.47164686324774,23.242986583600327],[77.47203310134589,23.2428781446916],[77.47211893203436,23.24237538132672],[77.4725588143128,23.242188076843604],[77.47259100082098,23.241902190546604],[77.47295578124701,23.242188076843604],[77.47293432357489,23.243400622259205],[77.47263391616522,23.24357806700535],[77.47251589896857,23.243785085577528],[77.47215111854254,23.24409068383501],[77.47203310134589,23.24450471906707],[77.47100313308417,23.244997616477278],[77.47038086059271,23.245165201181862],[77.47025211456,23.244997616477278]])

sentinel_AOI = sentinel.filterBounds(sarapangi_AOI)
               

Total Number of Images in Image Collection

In [6]:
print('Total number of images :', sentinel_AOI.size().getInfo())

Total number of images : 166


## **NDCI Calculation**

In [7]:
def addNDCI(image):
  ndci = image.normalizedDifference(['B5', 'B4']).rename('NDCI')
  return image.addBands(ndci)
with_ndci = sentinel_AOI.map(addNDCI)  

Calculating the Average NDCI on every image in the image collection

In [8]:
def meanNDCI(image):
  image = ee.Image(image)
  mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['NDCI']),
                           geometry = sarapangi_AOI,
                           scale = image.projection().nominalScale().getInfo(),
                           maxPixels = 100000,
                           bestEffort = True);
  return mean.get('NDCI').getInfo()

In [ ]:
Images_ndci = with_ndci.select('NDCI').toList(with_ndci.size())
ndci_coll = []
for i in range(Images_ndci.length().getInfo()):
  image = ee.Image(Images_ndci.get(i-1))
  temp_ndci = meanNDCI(image)
  ndci_coll.append(temp_ndci)

Getting Dates from the Image Collection

In [ ]:
dates = np.array(with_ndci.aggregate_array("system:time_start").getInfo())
day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

Creating the Dataframe

In [11]:
df = pd.DataFrame(ndci_coll, index = day, columns = ['Chlorophyll'])
df.index = pd.to_datetime(df.index, format="%Y/%m/%d")
df.sort_index(ascending = True, inplace = True)

In [12]:
df

,Chlorophyll
2018-12-18,0.096767
2018-12-23,0.307404
2018-12-28,0.381000
2019-01-02,0.371014
2019-01-07,0.348492
...,...
2022-12-02,0.086285
2022-12-07,0.092501
2022-12-17,0.098416
2022-12-22,0.101077


In [13]:
df.to_csv('Chlorophyll.csv')

## **NDTI Calculation**

In [14]:
def addNDTI(image):
  ndti = image.normalizedDifference(['B4', 'B3']).rename('NDTI')
  return image.addBands(ndti)
with_ndti = sentinel_AOI.map(addNDTI)

In [15]:
def meanNDTI(image):
  image = ee.Image(image)
  mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['NDTI']),
                           geometry = sarapangi_AOI,
                           scale = image.projection().nominalScale().getInfo(),
                           maxPixels = 100000,
                           bestEffort = True);
  return mean.get('NDTI').getInfo()

In [16]:
Images_ndti = with_ndti.select('NDTI').toList(with_ndti.size())
ndti_coll = []
for i in range(Images_ndti.length().getInfo()):
  image = ee.Image(Images_ndti.get(i-1))
  temp_ndti = meanNDTI(image)
  ndti_coll.append(temp_ndti)

In [17]:
dates = np.array(with_ndti.aggregate_array("system:time_start").getInfo())
day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

In [18]:
df1 = pd.DataFrame(ndti_coll, index = day, columns = ['Turbidity'])
df1.index = pd.to_datetime(df1.index, format="%Y/%m/%d")
df1.sort_index(ascending = True, inplace = True)

In [19]:
df1

,Turbidity
2018-12-18,-0.009233
2018-12-23,-0.148784
2018-12-28,-0.201117
2019-01-02,-0.170928
2019-01-07,-0.167122
...,...
2022-12-02,-0.026350
2022-12-07,-0.023317
2022-12-17,-0.010846
2022-12-22,-0.024933


In [20]:
df1.to_csv('Turbidity.csv')

## **NDSI Calculation**

In [21]:
def addNDSI(image):
  ndsi = image.normalizedDifference(['B11', 'B12']).rename('NDSI')
  return image.addBands(ndsi)
with_ndsi = sentinel_AOI.map(addNDSI)

In [22]:
def meanNDSI(image):
  image = ee.Image(image)
  mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['NDSI']),
                           geometry = sarapangi_AOI,
                           scale = image.projection().nominalScale().getInfo(),
                           maxPixels = 100000,
                           bestEffort = True);
  return mean.get('NDSI').getInfo()

In [25]:
Images_ndsi = with_ndsi.select('NDSI').toList(with_ndsi.size())
ndsi_coll = []
for i in range(Images_ndsi.length().getInfo()):
  image = ee.Image(Images_ndsi.get(i-1))
  temp_ndsi = meanNDSI(image)
  ndsi_coll.append(temp_ndsi)

KeyboardInterrupt: 

In [ ]:
dates = np.array(with_ndsi.aggregate_array("system:time_start").getInfo())
day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

In [ ]:
df2 = pd.DataFrame(ndsi_coll, index = day, columns = ['Salinty'])
df2.index = pd.to_datetime(df2.index, format="%Y/%m/%d")
df2.sort_index(ascending = True, inplace = True)

In [ ]:
df2

,Salinty
2018-12-18,0.101079
2018-12-23,0.203123
2018-12-28,0.155669
2019-01-02,0.170395
2019-01-07,0.157733
...,...
2022-12-02,0.082386
2022-12-07,0.075466
2022-12-17,0.082740
2022-12-22,0.085847


In [ ]:
df2.to_csv('Salinty.csv')

# **Dissloved Oxygen Calculation**

In [ ]:
def addDO(image):
  do = image.normalizedDifference(['B4', 'B5']).rename('DO')
  return image.addBands(do)
with_do = sentinel_AOI.map(addDO)

In [ ]:
def meanDO(image):
  image = ee.Image(image)
  mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['DO']),
                           geometry = sarapangi_AOI,
                           scale = image.projection().nominalScale().getInfo(),
                           maxPixels = 100000,
                           bestEffort = True);
  return mean.get('DO').getInfo()

In [ ]:
Images_do = with_do.select('DO').toList(with_do.size())
do_coll = []
for i in range(Images_do.length().getInfo()):
  image = ee.Image(Images_do.get(i-1))
  temp_do = meanDO(image)
  do_coll.append(temp_do)

In [ ]:
dates = np.array(with_do.aggregate_array("system:time_start").getInfo())
day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

In [ ]:
df4 = pd.DataFrame(do_coll, index = day, columns = ['Dissolved Oxygen'])
df4.index = pd.to_datetime(df4.index, format="%Y/%m/%d")
df4.sort_index(ascending = True, inplace = True)

In [ ]:
df4

,Dissolved Oxygen
2018-12-18,-0.098176
2018-12-23,-0.309838
2018-12-28,-0.386301
2019-01-02,-0.375912
2019-01-07,-0.353629
...,...
2022-12-02,-0.088461
2022-12-07,-0.094587
2022-12-17,-0.101059
2022-12-22,-0.103657


In [ ]:
df4.to_csv('Dissolved Oxygen.csv')

# **pH**

In [ ]:
def addpH(image):
  ph = ee.Image(8.339).subtract(ee.Image(0.827).multiply(image.select('B1').divide(image.select('B8')))).rename('PH')
  return image.addBands(ph)
with_pH = sentinel_AOI.map(addpH)

In [ ]:
def meanpH(image):
  image = ee.Image(image)
  mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['PH']),
                           geometry = sarapangi_AOI,
                           scale = image.projection().nominalScale().getInfo(),
                           maxPixels = 100000,
                           bestEffort = True);
  return mean.get('PH').getInfo()

In [ ]:
Images_ph = with_pH.select('PH').toList(with_pH.size())
ph_coll= []
for i in range(Images_ph.length().getInfo()):
  image = ee.Image(Images_ph.get(i-1))
  temp_ph = meanpH(image)
  ph_coll.append(temp_ph)

In [ ]:
dates = np.array(with_pH.aggregate_array("system:time_start").getInfo())
day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

In [ ]:
df6 = pd.DataFrame(ph_coll, index = day, columns = ['pH'])
df6.index = pd.to_datetime(df6.index, format="%Y/%m/%d")
df6.sort_index(ascending = True, inplace = True)

In [ ]:
df6

,pH
2018-12-18,7.838490
2018-12-23,8.051001
2018-12-28,8.144490
2019-01-02,8.145825
2019-01-07,8.117733
...,...
2022-12-02,7.778258
2022-12-07,7.822361
2022-12-17,7.840244
2022-12-22,7.793080


In [ ]:
df6.to_csv('pH.csv')

# **Temperature**


Selecting the Satellite



In [ ]:
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").\
              filterDate("2016-01-01","2022-01-01")
sarapangi_AOI = ee.Geometry.Polygon([[77.47025211456,23.244997616477278],[77.4703594029206,23.244248411693995],[77.47089584472357,23.244061109840693],[77.4707027256745,23.24396252981228],[77.47074564101874,23.243775227557578],[77.47029502990424,23.243647073231774],[77.47043450477301,23.243499202702868],[77.47086365821539,23.243440054445415],[77.47074564101874,23.24331189979763],[77.47107823493658,23.243035873984248],[77.47135718467413,23.243213319215556],[77.47164686324774,23.242986583600327],[77.47203310134589,23.2428781446916],[77.47211893203436,23.24237538132672],[77.4725588143128,23.242188076843604],[77.47259100082098,23.241902190546604],[77.47295578124701,23.242188076843604],[77.47293432357489,23.243400622259205],[77.47263391616522,23.24357806700535],[77.47251589896857,23.243785085577528],[77.47215111854254,23.24409068383501],[77.47203310134589,23.24450471906707],[77.47100313308417,23.244997616477278],[77.47038086059271,23.245165201181862],[77.47025211456,23.244997616477278]])

landsat_AOI = landsat.filterBounds(sarapangi_AOI)

In [ ]:
print('Total number of images :', landsat_AOI.size().getInfo())

Total number of images : 119


In [ ]:
def addtemp(image):
  temp = image.select('ST_B.*').multiply(0.00341802).add(149.0).subtract(273.15).rename('temperature')
  return image.addBands(temp)
with_temp = landsat_AOI.map(addtemp)


In [ ]:
def meantemp(image):
  image = ee.Image(image)
  mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['temperature']),
                           geometry = sarapangi_AOI,
                           scale = image.projection().nominalScale().getInfo(),
                           maxPixels = 100000,
                           bestEffort = True);
  return mean.get('temperature').getInfo()

In [ ]:
Images_temp = with_temp.select('temperature').toList(with_temp.size())
temp_coll = []
for i in range(Images_temp.length().getInfo()):
  image = ee.Image(Images_temp.get(i-1))
  tempe_temp = meantemp(image)
  temp_coll.append(tempe_temp)

In [ ]:
dates = np.array(with_temp.aggregate_array("system:time_start").getInfo())
day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

In [ ]:
df5 = pd.DataFrame(temp_coll, index = day, columns = ['Temperature'])
df5.index = pd.to_datetime(df5.index, format="%Y/%m/%d")
df5.sort_index(ascending = True, inplace = True)

In [ ]:
df5

,Temperature
2016-01-07,44.078925
2016-01-23,26.962331
2016-02-08,22.530777
2016-02-24,28.274133
2016-03-11,32.120028
...,...
2021-10-19,7.204958
2021-11-04,26.116928
2021-11-20,28.544911
2021-12-06,27.662679


In [ ]:
df5.to_csv('Temperature.csv')

# **Dissolved Oxygen Matter**


Selecting the Satellite

In [ ]:
sentinel3 = ee.ImageCollection("COPERNICUS/S3/OLCI").\
              filterDate("2018-06-02","2018-07-01")
sarapangi_AOI = ee.Geometry.Polygon([[77.47025211456,23.244997616477278],[77.4703594029206,23.244248411693995],[77.47089584472357,23.244061109840693],[77.4707027256745,23.24396252981228],[77.47074564101874,23.243775227557578],[77.47029502990424,23.243647073231774],[77.47043450477301,23.243499202702868],[77.47086365821539,23.243440054445415],[77.47074564101874,23.24331189979763],[77.47107823493658,23.243035873984248],[77.47135718467413,23.243213319215556],[77.47164686324774,23.242986583600327],[77.47203310134589,23.2428781446916],[77.47211893203436,23.24237538132672],[77.4725588143128,23.242188076843604],[77.47259100082098,23.241902190546604],[77.47295578124701,23.242188076843604],[77.47293432357489,23.243400622259205],[77.47263391616522,23.24357806700535],[77.47251589896857,23.243785085577528],[77.47215111854254,23.24409068383501],[77.47203310134589,23.24450471906707],[77.47100313308417,23.244997616477278],[77.47038086059271,23.245165201181862],[77.47025211456,23.244997616477278]])
sentinel3_AOI = sentinel3.filterBounds(sarapangi_AOI)

In [ ]:
print('Total number of images :', sentinel3_AOI.size().getInfo())

Total number of images : 363


In [ ]:
def addDM(image):
  rgb = image.select(['Oa08_radiance', 'Oa06_radiance', 'Oa04_radiance'])\
              .multiply(ee.Image([0.00876539, 0.0123538, 0.0115198]))
  dm = rgb.select('Oa08_radiance').divide(rgb.select('Oa04_radiance')).rename('dom')
  return image.addBands(dm)
with_dm = sentinel3_AOI.map(addDM)

In [ ]:
def meanDM(image):
  image = ee.Image(image)
  mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['dom']),
                           geometry = sarapangi_AOI,
                           scale = image.projection().nominalScale().getInfo(),
                           maxPixels = 100000,
                           bestEffort = True);
  return mean.get('dom').getInfo()

In [ ]:
Images_dm = with_dm.select('dom').toList(with_dm.size())
dm_coll= []
for i in range(Images_dm.length().getInfo()):
  image = ee.Image(Images_dm.get(i-1))
  temp_dm = meanDM(image)
  dm_coll.append(temp_dm)

In [ ]:
dates = np.array(with_dm.aggregate_array("system:time_start").getInfo())
day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

In [ ]:
df6 = pd.DataFrame(dm_coll, index = day, columns = ['Dissolved Oxygen Matter'])
df6.index = pd.to_datetime(df6.index, format="%Y/%m/%d")
df6.sort_index(ascending = True, inplace = True)

In [ ]:
df6

,Dissolved Oxygen Matter
2018-04-01,NaN
2018-04-02,0.580239
2018-04-03,0.540352
2018-04-03,NaN
2018-04-03,NaN
...,...
2018-04-30,NaN
2018-04-30,NaN
2018-04-30,NaN
2018-04-30,NaN


In [ ]:
df6.to_csv('DM_2018-04-02to2018-05-01.csv')

# **Suspended Matter**

In [ ]:
def addSM(image):
  rgb = image.select(['Oa08_radiance', 'Oa06_radiance', 'Oa04_radiance'])\
              .multiply(ee.Image([0.00876539, 0.0123538, 0.0115198]))
  suspended_matter = rgb.select('Oa08_radiance').divide(rgb.select('Oa06_radiance')).rename('suspended_matter')
  return image.addBands(suspended_matter)
with_sm = sentinel3_AOI.map(addSM)

In [ ]:
def meanSM(image):
  image = ee.Image(image)
  mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['suspended_matter']),
                           geometry = sarapangi_AOI,
                           scale = image.projection().nominalScale().getInfo(),
                           maxPixels = 100000,
                           bestEffort = True);
  return mean.get('suspended_matter').getInfo()

In [ ]:
Images_sm = with_sm.select('suspended_matter').toList(with_sm.size())
sm_coll= []
for i in range(Images_sm.length().getInfo()):
  image = ee.Image(Images_sm.get(i-1))
  temp_sm = meanSM(image)
  sm_coll.append(temp_sm)